In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Создаем объект для токенизапции
tokenizer = Tokenizer(num_words=1000)
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O /tmp/sonnets.txt
data = open('/tmp/sonnets.txt').read()
#Читаем текст
corpus = data.lower().split("\n")

--2020-07-14 07:19:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 74.125.204.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘/tmp/sonnets.txt’

/tmp/sonnets.txt    100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-07-14 07:19:07 (97.9 MB/s) - ‘/tmp/sonnets.txt’ saved [93578/93578]



In [3]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [4]:
callback = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 20, input_length=max_sequence_len-1))
model.add(LSTM(20))
model.add(Dense(total_words, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 20)            64220     
_________________________________________________________________
lstm (LSTM)                  (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 3211)              67431     
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              10313732  
Total params: 10,448,663
Trainable params: 10,448,663
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
 history = model.fit(predictors, label, epochs=100, validation_split=0.2, callbacks=[callback],verbose=1)

Epoch 1/100
322/322 [==============================] - 6s 19ms/step - loss: 6.2972 - accuracy: 0.0274 - val_loss: 6.0319 - val_accuracy: 0.0272
Epoch 2/100
322/322 [==============================] - 5s 17ms/step - loss: 5.8518 - accuracy: 0.0283 - val_loss: 5.9870 - val_accuracy: 0.0322
Epoch 3/100
322/322 [==============================] - 5s 17ms/step - loss: 5.8005 - accuracy: 0.0289 - val_loss: 6.0050 - val_accuracy: 0.0202
Epoch 4/100
322/322 [==============================] - 5s 17ms/step - loss: 5.7552 - accuracy: 0.0315 - val_loss: 5.9775 - val_accuracy: 0.0225
Epoch 5/100
322/322 [==============================] - 5s 16ms/step - loss: 5.7060 - accuracy: 0.0327 - val_loss: 5.9752 - val_accuracy: 0.0225
Epoch 6/100
322/322 [==============================] - 5s 16ms/step - loss: 5.6607 - accuracy: 0.0325 - val_loss: 5.9729 - val_accuracy: 0.0221
Epoch 7/100
322/322 [==============================] - 5s 16ms/step - loss: 5.6230 - accuracy: 0.0352 - val_loss: 6.0173 - val_accuracy:

In [ ]:
#seed_text = "Help me Obi Wan Kenobi, you're my only hope"
# seed_text = "She is my girl, I see in my hend"
seed_text = "She is my love, forever"
next_words = 100
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
  #y_p=model.predict(token_list)
	output_word = ""
	y_p=model.predict(token_list)
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
She is my love, forever of my love of thee my love doth be be be my love of my love of thee my love doth be be be my love of my love of thee my love doth be be be my love of my love of thee my love doth be be be my love of my love of thee my love doth be be be my love of my love of thee my love doth be be be my love of my love of thee my love doth be be be my love of my love of thee my love doth be
